In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import time

camera = cv2.VideoCapture(0)
model = YOLO("yolov8n.pt")

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_LEFT)
percentage_bar_annotator = sv.BoundingBoxAnnotator()

while True:
    success, frame = camera.read() 
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    coord_center = []
    for i in range (len(detections)):
        centers = (detections.xyxy[:,:2][i]+detections.xyxy[:,2:][i])/2
        coord_center.append(centers)
    coord_center=np.array(coord_center)  
    if coord_center.size != 0:
        print(coord_center)
        cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    frame = label_annotator.annotate(
        scene=frame,
        detections=detections
    )
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
Using TensorFlow backend.


[[     400.92      384.39]]
[[     400.94      386.24]]
[[     400.67         384]]
[[     400.89      386.03]]
[[     400.84      385.24]]
[[     400.72      385.39]]
[[     401.18      387.22]]
[[     400.89      384.94]
 [     499.85      214.39]]
[[        401      385.76]
 [     499.49      214.36]]
[[     401.14      386.13]]
[[     400.83      384.64]]
[[     401.05       384.5]
 [     500.04      214.19]]
[[     401.02      385.44]]
[[     401.19      385.06]]
[[     401.09      386.55]
 [      499.8      214.15]]
[[     401.23      386.59]
 [     499.61      214.22]]
[[     401.05      385.14]
 [     499.76      214.38]]
[[     400.91       386.3]]
[[     401.19      385.23]]
[[     401.14      384.55]]
[[     400.91      385.36]
 [     499.55      214.31]]
[[        401      385.53]]
[[     400.98      386.85]
 [     500.15      214.31]]
[[     401.27      386.26]
 [     500.11      214.15]]
[[      400.9      386.09]]
[[     401.21       385.8]
 [     499.96      214.16]]
[[

[[     400.95      385.13]]
[[      400.7      385.47]]
[[     400.97      384.83]]
[[     400.87      385.06]]
[[      401.2      384.77]
 [     500.18      214.48]]
[[     400.91       385.5]]
[[     401.13      386.48]]
[[     400.79      386.23]]
[[     401.03      385.22]
 [     499.82       214.3]]
[[     401.07      385.39]
 [      499.6      214.38]]
[[      400.9      385.98]]
[[     400.91      385.53]]
[[     400.94      386.21]]
[[     401.15      385.78]
 [     499.51      214.33]]
[[     400.94      385.76]]
[[     400.87      386.44]]
[[     401.03      385.46]]
[[      400.8      385.45]
 [     499.89      214.34]]
[[     400.84      385.09]]
[[      401.1      386.47]]
[[     400.76      386.62]]
[[     400.81      385.63]
 [     499.68      214.19]]
[[     401.09      385.42]
 [     499.45      214.57]]
[[      401.1      385.88]]
[[     401.14      387.01]]
[[     400.92      385.08]]
[[     400.92      386.28]]
[[     401.09      384.69]
 [     499.45      214.34]]


[[     400.42      387.04]]
[[     400.43      386.75]]
[[     400.21      386.36]]
[[     400.18      387.83]]
[[     400.48         387]]
[[     400.27      387.84]]
[[     400.28       387.2]]
[[     400.33       385.3]]
[[     400.34      386.95]]
[[     400.42      386.39]
 [     498.91      214.37]]
[[     400.32      385.52]]
[[     400.38      385.72]]
[[     400.34       388.1]]
[[     400.39       387.1]]
[[     400.38      386.93]
 [      499.1      214.18]]
[[     400.42      385.95]]
[[     400.26      386.98]
 [     498.93      214.37]]
[[     400.44      388.48]]
[[     400.23       386.2]]
[[     400.23      386.91]]
[[      400.3      386.24]
 [     499.13      214.29]]
[[      400.4      386.36]]
[[      400.3      385.93]]
[[     400.35      386.47]]
[[     400.29      386.06]]
[[     400.42      386.42]]
[[     400.38      386.79]
 [     499.01      214.23]]
[[     400.53      386.54]]
[[     400.41      385.56]
 [     499.11      214.54]]
[[     400.32         388]

[[     400.32      387.88]]
[[     400.46      385.06]]
[[     400.45      388.02]]
[[     400.44      386.71]]
[[     400.37      385.88]]
[[     400.46      386.86]]
[[     400.25      388.29]]
[[     400.32      387.16]]
[[     400.42      387.84]]
[[     400.62      387.42]]
[[      400.3      386.33]]
[[     400.33      387.48]]
[[     400.33      385.66]]
[[     400.25      386.37]]
[[     400.25      385.88]]
[[     400.29      388.04]]
[[     400.42      385.49]]
[[     400.31      387.64]]
[[      400.4      386.62]]
[[     400.43      387.18]]
[[     400.22      386.47]]
[[     400.46      386.52]]
[[     400.52      386.32]]
[[     400.38      387.58]]
[[     400.49      388.12]]
[[     400.31      387.11]]
[[     400.51      387.61]]
[[     400.46       387.1]
 [      499.5      214.56]]
[[     400.31      385.78]]
[[     400.45      387.31]]
[[      400.3      386.85]]
[[     400.35      387.36]]
[[     400.33      387.13]]
[[     400.42      387.03]]
[[     400.42      38

...

[[     400.52      387.89]]
[[     400.38       384.9]]
[[     400.46      387.88]]
[[     400.43      386.01]
 [      499.4      214.58]]
[[     400.32      386.28]]
[[     400.26      386.54]]
[[     400.32      387.84]]
[[     400.45      385.95]]
[[     400.28      386.98]]
[[      400.5      387.65]]
[[     400.29      386.99]]
[[     400.48      387.11]]
[[     400.42      387.52]]
[[     400.25      386.91]]
[[     400.34      386.59]]
[[      400.5      387.85]
 [      498.9      214.46]]
[[     400.29       386.8]]
[[     400.34      387.12]
 [     499.33      214.46]]
[[     400.36      387.43]]
[[     400.37      387.22]]
[[      400.2       386.8]]
[[     400.44       387.4]]
[[      400.3      387.53]]
[[     400.34      386.44]]
[[     400.35      387.41]]
[[     400.45      387.61]]
[[      400.4      387.26]]
[[      400.2      386.82]]
[[     400.29      386.03]]
[[     400.42      386.87]]
[[     400.56      386.34]]
[[     400.49      387.74]]
[[     400.36       386

[[     400.26      385.25]
 [     498.99      214.37]]
[[     400.52      386.75]]
[[     400.38      387.28]]
[[     400.48      387.45]]
[[     400.46      386.66]]
[[     400.49      386.36]]
[[     400.25      386.56]]
[[     400.45      386.26]]
[[     400.36      387.38]]
[[     400.47      386.89]]
[[     400.23      387.74]]
[[     400.24      385.79]]
[[     400.32      387.41]]
[[     400.38      387.89]]
[[     400.47      387.71]]
[[     400.48      387.01]]
[[     400.36      386.08]]
[[     400.53      387.28]]
[[     400.31      387.32]]
[[     400.46      387.55]]
[[     400.43      386.31]]
[[     400.46      387.16]]
[[     400.37       387.4]]
[[     400.33       386.6]]
[[     400.55      387.99]]
[[     400.44      386.34]]
[[     400.25      387.14]]
[[     400.54      388.51]]
[[     400.43      387.51]]
[[     400.39      385.94]]
[[     400.25      388.09]]
[[     400.51      387.14]]
[[     400.29      387.45]]
[[     400.35      386.05]]
[[      400.4      38

[[     326.04      366.92]
 [     243.85      91.557]
 [      198.4      77.835]]
[[     326.72      369.95]
 [      196.4      79.513]
 [     168.24      47.737]
 [     241.98       92.85]]
[[     326.51      370.06]
 [     241.61      88.053]
 [     241.29      88.367]
 [     165.46      43.636]]
[[     326.53       369.8]
 [     246.44      40.618]]
[[     326.47      369.95]]
[[      326.6       369.7]]
[[     326.58      369.72]]
[[     326.89      370.15]]
[[     326.78      370.28]]
[[     326.72      370.22]]
[[      326.7      370.19]]
[[     326.68      370.04]]
[[     326.72      370.03]]
[[      326.7      370.18]]
[[     326.55      370.25]]
[[     326.61      370.07]]
[[      326.6      370.06]]
[[     326.62       370.1]]
[[     326.61      370.08]]
[[     326.66      370.15]]
[[     326.67      370.12]]
[[     326.51       370.1]]
[[     326.75      370.32]]
[[      326.7      370.04]]
[[     326.63      370.21]]
[[     326.58      369.99]]
[[     326.44      370.27]]
[

[[     326.62      369.96]]
[[     326.62      370.23]]
[[     326.55       370.1]]
[[     326.79      370.38]]
[[     326.55      370.04]]
[[     326.64      370.13]]
[[     326.62      369.98]]
[[      326.6      370.11]]
[[     326.66      370.21]]
[[     326.74      370.25]]
[[     326.71      370.13]]
[[     326.66      369.93]]
[[     326.82      370.24]]
[[     326.62      370.13]]
[[     326.58       370.2]]
[[      326.5      370.19]]
[[     326.58      370.18]]
[[     326.74      370.15]]
[[      326.6      370.05]]
[[     326.64      370.25]]
[[     326.52      370.07]]
[[     326.83      370.19]]
[[     326.79      370.25]]
[[      326.7      369.94]]
[[     326.47      370.09]]
[[      326.7      370.15]]
[[     326.55       370.2]]
[[     326.67      370.18]]
[[     326.59      370.24]]
[[     326.58      370.34]]
[[      326.6      370.03]]
[[     326.51      370.18]]
[[     326.64       370.1]]
[[     326.74      370.14]]
[[     326.67      370.01]]
[[     326.55      3

[[      326.7      369.98]]
[[     326.62      370.23]]
[[     326.46      370.08]]
[[     326.49      370.19]]
[[     326.67      370.16]]
[[     326.57      370.19]]


In [1]:
import cv2
from realsense_camera import *
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import pyrealsense2
from kuka import Kuka
import time
from openshowvar import *


#Connect robot
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)

kuka.read_cartessian()
x0=282.73
y0=122.48
z0=451.23
# Построение траектории
trajectory_arr = []
# Построение квадрата
trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous(trajectory_arr)

fx=604.602
fy=604.162
Cx=320
Cy=240
cap = RealsenseCamera()
#camera = cv2.VideoCapture(0)
model = YOLO("yolov8n.pt")

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_LEFT)
percentage_bar_annotator = sv.BoundingBoxAnnotator()

while True:
    ret, frame, depth = cap.get_frame_stream()
    #success, frame = camera.read() 
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    #classes = result[0].boxes.cls.tolist() #Класс
    #print(classes)
    coord_center = []
    for i in range (len(detections)):
        centers = (detections.xyxy[:,:2][i]+detections.xyxy[:,2:][i])/2
        coord_center.append(centers)
    coord_center=np.array(coord_center)  
    if coord_center.size != 0:
        print(coord_center)
        cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
        for i in range(len(detections)):
            X_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cx-coord_center[0][0])/fx)
            Y_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cy-coord_center[0][1])/fy)
            X_mm=X_mm+30
            Y_mm=Y_mm+65
            Z_mm=depth[int(coord_center[0][1]), int(coord_center[0][0])]-40-50
            print('Xmm',X_mm)
            print('Ymm',Y_mm)
            print('Zmm',Z_mm)
            '''
            kuka.open_grip()
            kuka.read_cartessian()
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([kuka.x_cartessian-X_mm, kuka.y_cartessian+Y_mm,kuka.z_cartessian-Z_mm,90.57,0, -180]))
            trajectory_arr = np.array(trajectory_arr)
            kuka.lin_continuous(trajectory_arr)
            kuka.close_grip()
            time.sleep(1)
            kuka.read_cartessian()
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, -180]))
            trajectory_arr = np.array(trajectory_arr)
            kuka.lin_continuous(trajectory_arr)
            
            kuka.read_cartessian()
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([kuka.x_cartessian-X_mm, kuka.y_cartessian+Y_mm,kuka.z_cartessian-Z_mm,90.57,0, -180]))
            trajectory_arr = np.array(trajectory_arr)
            kuka.lin_continuous(trajectory_arr)
            kuka.open_grip()
            kuka.read_cartessian()
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, -180]))
            trajectory_arr = np.array(trajectory_arr)
            kuka.lin_continuous(trajectory_arr)
            '''
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    frame = label_annotator.annotate(
        scene=frame,
        detections=detections
    )
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
Using TensorFlow backend.


[[     191.04      43.786]]
Xmm 126.40953695943132
Ymm 211.7962589839945
Zmm 362
[[      204.6      98.268]]
Xmm 107.109975585566
Ymm 159.77558331910694
Zmm 314
[[     214.05      97.878]]
Xmm 101.67427495840788
Ymm 161.21266993458306
Zmm 319
[[     216.52      96.296]]
Xmm 102.22436159529457
Ymm 165.3752608828071
Zmm 332
[[     217.79      94.195]]
Xmm 100.32755703855595
Ymm 165.3951438657233
Zmm 326
[[     213.62      96.072]]
Xmm 99.67859405324056
Ymm 159.33815405806928
Zmm 306
[[     193.51      99.613]
 [     193.49      99.444]]
Xmm 103.22305855373466
Ymm 146.32823923242867
Zmm 260
Xmm 103.22305855373466
Ymm 146.32823923242867
Zmm 260
[[     291.26      137.83]]
Xmm 46.449444536124076
Ymm 123.50988967418016
Zmm 256
[[     202.04      133.76]]
Xmm 137.50538885748585
Ymm 161.89224572352356
Zmm 461
[[     210.24      130.91]]
Xmm 129.66199440416517
Ymm 164.12680506836068
Zmm 459
[[     205.86      235.76]]
Xmm 133.45701266694607
Ymm 68.84617922350503
Zmm 458
[[     273.23      124.3

[[     306.91       343.1]]
Xmm 41.66717066840498
Ymm -26.98383935160072
Zmm 449
[[     306.94      342.53]]
Xmm 41.61980815963177
Ymm -26.29964580911762
Zmm 448
[[     307.02      342.98]]
Xmm 41.5473021469599
Ymm -26.705377323970524
Zmm 448
[[     306.91      343.14]]
Xmm 41.65244944323911
Ymm -26.843483657425494
Zmm 448
[[     307.16      343.32]]
Xmm 41.44968362233608
Ymm -27.174667081641857
Zmm 449
[[     306.92       343.9]]
Xmm 41.656369773072306
Ymm -27.693460403932292
Zmm 449
[[     307.03      342.89]]
Xmm 41.538938157483145
Ymm -26.618143689497813
Zmm 448
[[     307.18      342.97]]
Xmm 41.41160456444255
Ymm -26.6896154834739
Zmm 448
[[     307.19      343.16]]
Xmm 41.40218149837621
Ymm -26.865767638817275
Zmm 448
[[     306.92       343.7]]
Xmm 41.642836286502835
Ymm -27.343487283800485
Zmm 448
[[     307.28       342.9]]
Xmm 41.316423450429475
Ymm -26.631622780681127
Zmm 448
[[     307.05      343.52]]
Xmm 41.52519731762848
Ymm -27.180162005550926
Zmm 448
[[     306.99    

Xmm 56.54766036944552
Ymm 14.006678487199785
Zmm 431
Xmm 56.54766036944552
Ymm 14.006678487199785
Zmm 431
[[        503      374.33]
 [     288.69      298.93]]
Xmm -88.64959869131884
Ymm -22.15971364313296
Zmm 302
Xmm -88.64959869131884
Ymm -22.15971364313296
Zmm 302
[[     505.78      370.37]
 [     288.01      298.82]]
Xmm -91.06758542755064
Ymm -20.019696473528356
Zmm 304
Xmm -91.06758542755064
Ymm -20.019696473528356
Zmm 304
[[     514.65      354.58]
 [     288.23      299.28]]
Xmm -95.55786049225101
Ymm -8.96686342346662
Zmm 300
Xmm -95.55786049225101
Ymm -8.96686342346662
Zmm 300
[[     516.71      344.22]
 [     286.28      298.22]]
Xmm -95.2641123604092
Ymm -1.4156189099206387
Zmm 295
Xmm -95.2641123604092
Ymm -1.4156189099206387
Zmm 295
[[     286.86      298.94]
 [     520.48      322.17]]
Xmm 58.50052531975581
Ymm 14.268141275901584
Zmm 430
Xmm 58.50052531975581
Ymm 14.268141275901584
Zmm 430
[[     287.16      298.81]
 [      533.4      305.41]]
Xmm 58.41102254618211
Ymm 

Xmm 57.98168540458744
Ymm 14.25595570225267
Zmm 432
[[     287.75      299.03]]
Xmm 57.84130218295982
Ymm 14.001958623156696
Zmm 432
[[      287.5      298.85]]
Xmm 58.00891985537981
Ymm 14.254609803521632
Zmm 431
[[     287.66      298.71]]
Xmm 57.974071632218255
Ymm 14.173406825813721
Zmm 433
[[     287.58      298.81]]
Xmm 57.99312052436717
Ymm 14.186372156967266
Zmm 432
[[     287.77      299.08]]
Xmm 57.76966313718829
Ymm 14.053390947285898
Zmm 431
[[     287.66      298.84]]
Xmm 57.918976683214744
Ymm 14.161824115792314
Zmm 432
[[     287.72      298.76]]
Xmm 57.820522940769564
Ymm 14.329007620233433
Zmm 431
[[     287.49      298.88]]
Xmm 58.01772959384917
Ymm 14.227003397532712
Zmm 431
[[     287.64      298.89]]
Xmm 57.88731916677598
Ymm 14.214608147560568
Zmm 431
[[     287.57       298.7]]
Xmm 57.99762606695319
Ymm 14.282876808632508
Zmm 432
[[     287.73      298.62]]
Xmm 57.911269177873166
Ymm 14.252871879267929
Zmm 433
[[     287.59      298.82]]
Xmm 58.03977794111483
Ymm

[[     287.63      299.07]]
Xmm 57.95083165974398
Ymm 13.961537412500192
Zmm 432
[[     287.63      298.91]]
Xmm 58.00200143326791
Ymm 14.00684194482011
Zmm 433
[[     287.62      298.87]]
Xmm 58.06691568088129
Ymm 13.942214458828921
Zmm 434
[[     288.03      298.81]]
Xmm 57.65393486725694
Ymm 14.089212967649303
Zmm 433
[[     287.73      298.74]]
Xmm 57.91702408962063
Ymm 14.152483846313807
Zmm 433
[[     287.66      298.89]]
Xmm 58.025152467940686
Ymm 13.926280472606066
Zmm 434
[[     287.82      298.78]]
Xmm 57.78375770922959
Ymm 14.21698469811026
Zmm 432
[[     287.68      298.96]]
Xmm 57.907436170625985
Ymm 14.058384840250582
Zmm 432
[[      287.6      298.78]]
Xmm 57.97077724885286
Ymm 14.21637824811345
Zmm 432
[[      287.7      298.96]]
Xmm 57.83488149958826
Ymm 14.15189502509002
Zmm 431
[[     287.54       299.1]]
Xmm 58.025001849098665
Ymm 13.936171982199113
Zmm 432
[[     287.86      298.84]]
Xmm 57.746712136855635
Ymm 14.165093672296813
Zmm 432
[[     287.46      298.86]]


[[     502.86      280.59]
 [     287.79      298.62]]
Xmm -80.69581478516807
Ymm 40.40876600758054
Zmm 276
Xmm -80.69581478516807
Ymm 40.40876600758054
Zmm 276
[[     502.51      281.07]
 [     287.55      298.81]]
Xmm -80.78434141337118
Ymm 40.05334812219756
Zmm 277
Xmm -80.78434141337118
Ymm 40.05334812219756
Zmm 277
[[      502.9      282.78]
 [     287.74      298.65]]
Xmm -81.3271235664123
Ymm 38.94140593965278
Zmm 278
Xmm -81.3271235664123
Ymm 38.94140593965278
Zmm 278
[[     502.73      283.38]
 [     287.94      298.67]]
Xmm -81.22429248910854
Ymm 38.5769968749069
Zmm 278
Xmm -81.22429248910854
Ymm 38.5769968749069
Zmm 278
[[     503.09      283.43]
 [     287.91      298.55]]
Xmm -81.7427180510349
Ymm 38.47593197788838
Zmm 279
Xmm -81.7427180510349
Ymm 38.47593197788838
Zmm 279
[[     503.78      284.21]
 [     287.95      298.57]]
Xmm -82.16451006691761
Ymm 38.00114027510614
Zmm 279
Xmm -82.16451006691761
Ymm 38.00114027510614
Zmm 279
[[     504.47      287.63]
 [     288.15

[[     287.56      298.91]]
Xmm 58.05828341421105
Ymm 14.003222692053079
Zmm 433
[[     287.43      298.85]]
Xmm 58.17591592222355
Ymm 14.052307156586963
Zmm 433
[[      287.5      298.87]]
Xmm 58.109998423905054
Ymm 14.03925671230293
Zmm 433
[[      287.3       298.9]]
Xmm 58.229938166257
Ymm 14.108825659549929
Zmm 432
[[     287.71      298.84]]
Xmm 57.87768905027734
Ymm 14.158554559287808
Zmm 432
[[     287.37      298.91]]
Xmm 58.223275150090345
Ymm 13.999814782513326
Zmm 433
[[     287.37      298.82]]
Xmm 58.22945244068167
Ymm 14.082185805342519
Zmm 433
[[     287.59      298.88]]
Xmm 57.927212818292425
Ymm 14.225424384797407
Zmm 431
[[     287.42      298.85]]
Xmm 58.180430096117206
Ymm 14.052597753524466
Zmm 433
[[     287.57      298.95]]
Xmm 58.05310927337388
Ymm 13.968377477456634
Zmm 433
[[     287.59      298.92]]
Xmm 58.03724366805172
Ymm 13.995455828450837
Zmm 433
[[     287.59      298.82]]
Xmm 58.08863043567814
Ymm 13.988454780974003
Zmm 434
[[     287.41      298.65]]

Xmm 53.51629747880631
Ymm 13.917582971427521
Zmm 432
[[     292.81      299.14]]
Xmm 53.477249443060785
Ymm 13.903212743242399
Zmm 432
[[     292.84      299.23]]
Xmm 53.45084854439884
Ymm 13.823846895834642
Zmm 432
[[     292.74      299.11]]
Xmm 53.57887657913801
Ymm 13.833434826843579
Zmm 433
[[     292.93      299.24]]
Xmm 53.27918586770201
Ymm 14.01356765005702
Zmm 430
[[     292.89      299.08]]
Xmm 53.44902148333157
Ymm 13.856761835553712
Zmm 433
[[     292.79      298.98]]
Xmm 53.45162899632088
Ymm 14.140762985306118
Zmm 431
[[     292.88      299.06]]
Xmm 53.41435628427229
Ymm 13.975248455906183
Zmm 432
[[        293      299.04]]
Xmm 53.309516787489954
Ymm 13.991306197125887
Zmm 432
[[     292.87      299.15]]
Xmm 53.4202319333757
Ymm 13.893562278075876
Zmm 432
[[     292.88      299.15]]
Xmm 53.368370393353814
Ymm 13.994125335954102
Zmm 431
[[     292.82      299.22]]
Xmm 53.5131438715637
Ymm 13.732676943241735
Zmm 433
[[      292.9      299.06]]
Xmm 53.44577444596947
Ymm 13

KeyboardInterrupt: 

In [1]:
import cv2
from realsense_camera import *
from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import pyrealsense2
from kuka import Kuka
import time
from openshowvar import *


#Connect robot
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.set_base(8)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)

kuka.read_cartessian()
x0=282.73
y0=122.48
z0=451.23
# Построение траектории
trajectory_arr = []
# Построение квадрата
trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous(trajectory_arr)

fx=604.602
fy=604.162
Cx=320
Cy=240
cap = RealsenseCamera()
#camera = cv2.VideoCapture(0)
model = YOLO("yolov8n-seg.pt")

def getOrientation(pts, img):
    rect = cv2.minAreaRect(pts)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = int(rect[2])
    if width < height:
        angle = 90- angle
    else:
        angle = 90+(90-angle)
    return angle,width, height

while True:
    ret, frame, depth = cap.get_frame_stream()
    #success, frame = camera.read() 
    results = model(frame, verbose=False)
    boxes = results[0].boxes.xyxy.tolist() #Рамка
    #print(boxes)
    if boxes!=[]:
        frame=results[0].plot()
        classes = results[0].boxes.cls.tolist() #Класс
        #print(classes)
        confidences = results[0].boxes.conf.tolist()
        #print(confidences)
        mask_imshow = results[0].masks[0].data.cpu().numpy().transpose(1, 2, 0)#Маска двухбитная 
        mask1 = mask_imshow.astype(np.uint8)
        contours, _ = cv2.findContours(mask1, cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_NONE) 
        if len(contours) != 0: 
            maxc = max(contours, key=cv2.contourArea)
            cv2.drawContours(frame, maxc, -1, (0, 0, 255), 2)
        rotation,width,height=getOrientation(maxc, frame)
        print(rotation)
        coord_center = []
        for i in range (len(boxes)):
            x = (boxes[i][0]+boxes[i][2])/2
            y= (boxes[i][1]+boxes[i][3])/2
            coord_center.append([x,y])
        coord_center=np.array(coord_center)  
        if coord_center.size != 0:
            print(coord_center)
            cv2.circle(frame, (int(coord_center[0][0]),int(coord_center[0][1])), 3, (118, 103, 154), 3)
            for i in range(len(boxes)):
                X_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cx-coord_center[0][0])/fx)
                Y_mm=(depth[int(coord_center[0][1]), int(coord_center[0][0])])*((Cy-coord_center[0][1])/fy)
                X_mm=X_mm+30
                Y_mm=Y_mm+65
                Z_mm=depth[int(coord_center[0][1]), int(coord_center[0][0])]-40-50
                print('Xmm',X_mm)
                print('Ymm',Y_mm)
                print('Zmm',Z_mm)
                kuka.open_grip()
                kuka.read_cartessian()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([kuka.x_cartessian-X_mm, kuka.y_cartessian+Y_mm,kuka.z_cartessian-Z_mm,rotation,0, -180]))
                trajectory_arr = np.array(trajectory_arr)
                kuka.lin_continuous(trajectory_arr)
                kuka.close_grip()
                
                time.sleep(1)
                kuka.read_cartessian()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, -180]))
                trajectory_arr = np.array(trajectory_arr)
                kuka.lin_continuous(trajectory_arr)

                kuka.read_cartessian()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([kuka.x_cartessian-X_mm, kuka.y_cartessian+Y_mm,kuka.z_cartessian-Z_mm,90.57,0, -180]))
                trajectory_arr = np.array(trajectory_arr)
                kuka.lin_continuous(trajectory_arr)
                kuka.open_grip()
                kuka.read_cartessian()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([x0, y0, z0, 90.57, 0, -180]))
                trajectory_arr = np.array(trajectory_arr)
                kuka.lin_continuous(trajectory_arr)
                    
    cv2.imshow("mask_imshow", mask_imshow)
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

Using TensorFlow backend.


65
[[     291.13      299.33]]
Xmm 54.924720869325824
Ymm 13.735595237604166
Zmm 432


error: 'H' format requires 0 <= number <= 65535